Importing the Libraries

In [19]:
import cv2
import cvzone
import os
import random
from cvzone.FaceMeshModule import FaceMeshDetector
import tkinter as tk
from tkinter import Label, Button, Canvas
from PIL import Image, ImageTk
import numpy as np
import mediapipe as mp  

print("All libraries are installed successfully!")


All libraries are installed successfully!


pip install cv2 cvzone os random tkinter pillow numpy

Start the webcam

In [20]:
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    root = tk.Tk()
    root.title("Error")
    Label(root, text="Error: Could not open webcam.", font=("Arial", 20), fg="red").pack(pady=20)
    Button(root, text="Exit", command=root.quit, font=("Arial", 14), bg="red", fg="white").pack(pady=10)
    root.mainloop()
    exit()

cap.set(3, 800)
cap.set(4, 600)

True

Face Mesh Detection

In [21]:
# Face mesh detector
detector = FaceMeshDetector(staticMode=False, maxFaces=1, minDetectionCon=0.5, minTrackCon=0.5)

# Mouth detection landmarks
idList = [0, 17, 78, 292]

Load the Data

In [22]:
from pathlib import Path


In [23]:
import os
from pathlib import Path
import cv2

base_path = Path().resolve()  # Get the directory where the notebook is located

folderEatable = base_path / "eatable"
folderNonEatable = base_path / "noneatable"

# Check the "non_eatable" folder exists
if not folderNonEatable.exists():
    print(f"Error: Folder '{folderNonEatable}' not found. Please create the folder and add images.")
    listNonEatable = []  # Set an empty list to prevent further errors
else:
    listNonEatable = os.listdir(str(folderNonEatable))

# Check the "eatable" folder exists
if not folderEatable.exists():
    print(f"Error: Folder '{folderEatable}' not found. Please create the folder and add images.")
    listEatable = []
else:
    listEatable = os.listdir(str(folderEatable))

# Load images 
eatables = [cv2.imread(str(folderEatable / obj), cv2.IMREAD_UNCHANGED) for obj in listEatable] if listEatable else []
nonEatables = [cv2.imread(str(folderNonEatable / obj), cv2.IMREAD_UNCHANGED) for obj in listNonEatable] if listNonEatable else []


Variables used in the Game

In [24]:
pos = [200, 0]
speed = 5
count = 0
isEatable = True
gameOver = False
currentObject = eatables[0]
game_running = False

Reset objects

In [25]:
def resetObject():
    global currentObject, isEatable, pos
    pos[0] = random.randint(50, 750)
    pos[1] = 0
    randNo = random.randint(0, 2)
    if randNo == 0:
        currentObject = nonEatables[random.randint(0, len(nonEatables) - 1)]
        isEatable = False
    else:
        currentObject = eatables[random.randint(0, len(eatables) - 1)]
        isEatable = True


UI Setup

In [26]:
import tkinter as tk

def show_error():
    global root  # Declare root as global
    root = tk.Tk()  # Now assign it separately
    root.title("Error")
    root.mainloop()  # Run the Tkinter window


In [27]:
def setup_window():
    global root
    root = tk.Tk()
    root.title("Eatable & Non-Eatable Game")
    root.geometry("840x720")


In [28]:
setup_window()


In [29]:

root.title("Eatable & Non-Eatable Game")
root.geometry("840x720")

canvas = Canvas(root, width=800, height=600)
canvas.pack()

score_label = Label(root, text="Score: 0", font=("Arial", 14), fg="purple")
score_label.pack()

object_image_label = Label(root)
object_image_label.pack()

status_label = Label(root, text="Press 'Start' to begin", font=("Arial", 14), fg="blue")
status_label.pack()


Function to Start the game

In [30]:
def start_game():
    global game_running, gameOver, count
    if not game_running:
        game_running = True
        gameOver = False
        count = 0
        resetObject()
        status_label.config(text="Game Started!")
        print("Start button clicked, game_running:", game_running)
        update_frame()

Function to Restart the game

In [31]:
def restart_game():
    global gameOver, count, game_running
    gameOver = False
    count = 0
    resetObject()
    status_label.config(text="Game Restarted!")
    if not game_running:
        game_running = True
        update_frame()


Function to Exit the game

In [32]:
def exit_game():
    global game_running
    game_running = False  # Stop the update_frame loop
    try:
        cap.release()  # Release the webcam
        print("Webcam released")
    except Exception as e:
        print(f"Error releasing webcam: {e}")
    try:
        root.quit()  # Stop the Tkinter main loop
        root.destroy()  # Destroy the window
        print("Window closed")
    except Exception as e:
        print(f"Error closing window: {e}")

Buttons Creation

In [33]:
start_button = Button(root, text="Start", command=start_game, font=("Arial", 16), bg="green", fg="white", padx=10, pady=5)
start_button.pack(side=tk.LEFT, padx=10, pady=5)

restart_button = Button(root, text="Restart", command=restart_game, font=("Arial", 16), bg="orange", fg="white", padx=10, pady=5)
restart_button.pack(side=tk.LEFT, padx=10, pady=5)

exit_button = Button(root, text="Exit", command=exit_game, font=("Arial", 16), bg="red", fg="white", padx=10, pady=5)
exit_button.pack(side=tk.LEFT, padx=10, pady=5)

Working once the Game starts

In [34]:
def update_frame():
    global pos, count, gameOver, currentObject, game_running
    if game_running:  # Only run if game_running is True
        success, img = cap.read()
        if success:
            img = cv2.resize(img, (800, 600))
            if not gameOver:
                img, faces = detector.findFaceMesh(img, draw=False)
                img = cvzone.overlayPNG(img, currentObject, pos)
                pos[1] += speed
                if pos[1] > 540:
                    resetObject()
                if faces:
                    face = faces[0]
                    up, down = face[idList[0]], face[idList[1]]
                    up_down, _ = detector.findDistance(up, down)
                    left_right, _ = detector.findDistance(face[idList[2]], face[idList[3]])
                    ratio = int((up_down / left_right) * 100)
                    cx, cy = (up[0] + down[0]) // 2, (up[1] + down[1]) // 2
                    distMouthObject, _ = detector.findDistance((cx, cy), (pos[0] + 25, pos[1] + 25))
                    mouthStatus = "OPEN" if ratio > 60 else "CLOSED"
                    cv2.putText(img, mouthStatus, (20, 30), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 255), 1)
                    if distMouthObject < 50 and ratio > 60:
                        if isEatable:
                            count += 1
                            resetObject()
                        else:
                            gameOver = True
                score_label.config(text=f"Score: {count}")
                cv2.putText(img, f'Score: {count}', (700, 30), cv2.FONT_HERSHEY_PLAIN, 1.5, (255, 0, 255), 2)
            else:
                cv2.putText(img, "GAME OVER", (200, 300), cv2.FONT_HERSHEY_PLAIN, 5, (255, 0, 255), 6)
                status_label.config(text="Game Over! Press 'Restart' to play again")
            
            # Update the falling object image
            object_img = cv2.resize(currentObject, (50, 50))
            if object_img.shape[2] == 4:
                object_img = cv2.cvtColor(object_img, cv2.COLOR_BGRA2RGBA)
            else:
                object_img = cv2.cvtColor(object_img, cv2.COLOR_BGR2RGB)
            object_img = Image.fromarray(object_img)
            object_img_tk = ImageTk.PhotoImage(image=object_img)
            object_image_label.config(image=object_img_tk)
            object_image_label.image = object_img_tk

            # Update the canvas
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(img)
            img_tk = ImageTk.PhotoImage(image=img)
            canvas.create_image(0, 0, anchor=tk.NW, image=img_tk)
            canvas.img_tk = img_tk
        else:
            img = np.zeros((600, 800, 3), dtype=np.uint8)
            cv2.putText(img, "Webcam Error", (200, 300), cv2.FONT_HERSHEY_PLAIN, 4, (255, 255, 255), 4)
            img = Image.fromarray(img)
            img_tk = ImageTk.PhotoImage(image=img)
            canvas.create_image(0, 0, anchor=tk.NW, image=img_tk)
            canvas.img_tk = img_tk
            print("Failed to read frame from webcam")
        if game_running:  # Only schedule next update if still running
            root.after(10, update_frame)

Calling the function

In [35]:
root.mainloop()
cap.release()  # Final cleanup
cv2.destroyAllWindows()

Webcam released
Window closed


Versions of the Libraries